In [1]:
import xarray as xr
import numpy as np
import xgcm
from xmitgcm import open_mdsdataset

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 16

from bbl_utils import *

# Progress bar for dask stuff
#from dask.diagnostics import ProgressBar
#ProgressBar().register()

import warnings
warnings.filterwarnings('ignore', '.*.')

nancol = (0.8,0.8,0.8)

div_cmap = plt.get_cmap('RdBu_r')
div_cmap.set_bad(color=nancol)

cmap = plt.get_cmap('viridis')
cmap.set_bad(color=nancol)

In [4]:
n = 5
diter = 28800
iter0 = 3182400
iters = np.arange(iter0, iter0+n*diter, diter)
iters

array([3182400, 3211200, 3240000, 3268800, 3297600])

In [3]:
iters = [3173760]

### Physical parameters

In [4]:
Γ = 0.0008613659531090722
θ = 1.53E-3
f = -0.53e-4

### Test grid pre-processing

In [13]:
data_dir_2D = f"../../MITgcm/experiments/rotated_BBTRE_rest-tracer/run/"
ds = open_mdsdataset(data_dir_2D,iters=iters,ignore_unknown_vars=True,prefix=["tracerDiag"], delta_t=60.)

In [14]:
def parallel_combine(ds_list, concat_dims):
    ds_new_list = []
    for concat_dim in concat_dims:
        tmp_list = []
        for ds in ds_list:
            tmp = ds.copy()
            for var in (list(ds.data_vars) + list(ds.coords)):
                if (concat_dim not in ds[var].dims) & any([(dim in concat_dims) for dim in ds[var].dims]):
                    tmp = tmp.drop_vars(var)
            tmp_list.append(tmp)
        ds_new_list.append(xr.combine_by_coords(tmp_list))
    return xr.merge(ds_new_list)

In [15]:
def periodic_extend(ds, concat_dims, dim_length, extend_multiples):
    ds_list = []
    for extend in range(extend_multiples[0], extend_multiples[1]+1):
        tmp = ds.copy()
        tmp_attrs = [tmp[dim].attrs for dim in concat_dims]
        for dim in concat_dims:
            tmp[dim] = tmp[dim] + extend*dim_length
        ds_list.append(tmp)
    ds = parallel_combine(ds_list, concat_dims = concat_dims)
    for i, dim in enumerate(concat_dims):
        ds[dim].attrs = tmp_attrs[i]
    return ds

In [16]:
dx = 500.
Lx = ds['XC'].size*dx
extend_multiples = [-1, 0]
ds = periodic_extend(ds, ['XC', 'XG'], Lx, extend_multiples)

In [17]:
H = ds['Depth'].max().values.copy().astype('>f8')
ds['Z'] += H
ds['Zl'] += H
ds['Zu'] += H
ds['Zp1'] += H

grid = Grid(ds, periodic=['X', 'Y'])

ds = ds.assign_coords({
    'Zr': (['XC', 'Z'], (ds['XC']*np.sin(θ) + ds['Z']*np.cos(θ)).values),
    'Xr': (['XC', 'Z'], (ds['XC']*np.cos(θ) - ds['Z']*np.sin(θ)).values),
    'Zr_V': (['XC', 'Z'], (ds['XC']*np.sin(θ) + ds['Z']*np.cos(θ)).values),
    'Xr_V': (['XC', 'Z'], (ds['XC']*np.cos(θ) - ds['Z']*np.sin(θ)).values),
    'Zr_U': (['XG', 'Z'], (ds['XG']*np.sin(θ) + ds['Z']*np.cos(θ)).values),
    'Xr_U': (['XG', 'Z'], (ds['XG']*np.cos(θ) - ds['Z']*np.sin(θ)).values),
    'Zr_W': (['XC', 'Zl'], (ds['XC']*np.sin(θ) + ds['Zl']*np.cos(θ)).values),
    'Xr_W': (['XC', 'Zl'], (ds['XC']*np.cos(θ) - ds['Zl']*np.sin(θ)).values)
})

ds['Depthr'] = ds['Depth'] - ds['XC']*np.tan(θ)

volume = (ds.drF * ds.rA * ds.hFacC)
day2seconds = (24*60*60)**-1

In [18]:
ds

<xarray.Dataset>
Dimensions:  (XC: 2400, XG: 2400, YC: 120, YG: 120, Z: 260, Zl: 260, Zp1: 261, Zu: 260, time: 1)
Coordinates:
    drF      (Z) float64 71.38 69.47 67.61 65.8 64.04 ... 5.0 5.0 5.0 5.0 5.0
    PHrefF   (Zp1) float64 0.0 700.3 1.382e+03 ... 3.218e+04 3.223e+04 3.228e+04
    iter     (time) int64 3173760
    drC      (Zp1) float64 35.69 70.43 68.54 66.71 64.92 ... 5.0 5.0 5.0 5.0 2.5
    PHrefC   (Z) float64 350.1 1.041e+03 1.713e+03 ... 3.22e+04 3.225e+04
  * time     (time) timedelta64[ns] 2204 days
  * Zl       (Zl) >f8 3.285e+03 3.214e+03 3.144e+03 ... 10.0 5.0 -1.364e-12
  * Zp1      (Zp1) >f8 3.285e+03 3.214e+03 3.144e+03 ... 5.0 -1.364e-12 -5.0
  * Z        (Z) >f8 3.25e+03 3.179e+03 3.111e+03 3.044e+03 ... 7.5 2.5 -2.5
  * YG       (YG) >f8 0.0 500.0 1e+03 1.5e+03 ... 5.85e+04 5.9e+04 5.95e+04
  * Zu       (Zu) >f8 3.214e+03 3.144e+03 3.077e+03 ... 5.0 -1.364e-12 -5.0
  * YC       (YC) >f8 250.0 750.0 1.25e+03 ... 5.875e+04 5.925e+04 5.975e+04
  * XC       (XC) float64 -3.998e+05 -3.992e+05 ... 7.992e+05 7.998e+05
    rA       (YC, XC) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    dxG      (YG, XC) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    Depth    (YC, XC) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    dyC      (YG, XC) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    rAs      (YG, XC) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    hFacC    (Z, YC, XC) float64 dask.array<chunksize=(260, 120, 800), meta=np.ndarray>
    hFacS    (Z, YG, XC) float64 dask.array<chunksize=(260, 120, 800), meta=np.ndarray>
    maskC    (Z, YC, XC) bool dask.array<chunksize=(260, 120, 800), meta=np.ndarray>
    maskS    (Z, YG, XC) bool dask.array<chunksize=(260, 120, 800), meta=np.ndarray>
  * XG       (XG) float64 -4e+05 -3.995e+05 -3.99e+05 ... 7.99e+05 7.995e+05
    dyG      (YC, XG) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    rAz      (YG, XG) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    dxC      (YC, XG) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    rAw      (YC, XG) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>
    hFacW    (Z, YC, XG) float64 dask.array<chunksize=(260, 120, 800), meta=np.ndarray>
    maskW    (Z, YC, XG) bool dask.array<chunksize=(260, 120, 800), meta=np.ndarray>
    Zr       (XC, Z) float64 2.638e+03 2.568e+03 ... 1.226e+03 1.221e+03
    Xr       (XC, Z) float64 -3.998e+05 -3.998e+05 ... 7.997e+05 7.997e+05
    Zr_V     (XC, Z) float64 2.638e+03 2.568e+03 ... 1.226e+03 1.221e+03
    Xr_V     (XC, Z) float64 -3.998e+05 -3.998e+05 ... 7.997e+05 7.997e+05
    Zr_U     (XG, Z) float64 2.638e+03 2.567e+03 ... 1.226e+03 1.221e+03
    Xr_U     (XG, Z) float64 -4e+05 -4e+05 -4e+05 ... 7.995e+05 7.995e+05
    Zr_W     (XC, Zl) float64 2.674e+03 2.602e+03 ... 1.229e+03 1.224e+03
    Xr_W     (XC, Zl) float64 -3.998e+05 -3.998e+05 ... 7.997e+05 7.997e+05
Data variables:
    TRAC01   (time, Z, YC, XC) float64 dask.array<chunksize=(1, 260, 120, 800), meta=np.ndarray>
    TRAC02   (time, Z, YC, XC) float64 dask.array<chunksize=(1, 260, 120, 800), meta=np.ndarray>
    Depthr   (YC, XC) float64 dask.array<chunksize=(120, 800), meta=np.ndarray>

In [20]:
ds.to_netcdf("../../MITgcm/experiments/rotated_BBTRE_rest-tracer/run/test_paraview.nc")